In [54]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd
from tqdm import tqdm
import seaborn as sns

from graspy.plot import heatmap
from graspy.utils import symmetrize
from graspy.simulations import sbm

from mgcpy.independence_tests.dcorr import DCorr
from mgcpy.independence_tests.rv_corr import RVCorr
from mgcpy.independence_tests.mgc import MGC

from simulations import rho_ER_marg, rho_sbm_marg, rho_sbm_diff_block
from utils import triu_no_diag, to_distance_mtx, identity, sbm_params, dcorr_power_two_sided, pearson_power_two_sided

In [69]:
def sample_rho_matrix(AL, BL, seed=0):
    np.random.seed(seed)
    if np.any(AL == 0) or np.any(AL == 1) \
            or np.any(BL == 0) or np.any(BL == 1):
        raise ValueError('block probabilities AL and BL cannot have 0 or 1')

    max_rho = np.minimum(np.sqrt(AL*(1-BL)/((1-AL)*BL)), np.sqrt((1-AL)*BL/(AL*(1-BL))))
    min_rho = np.maximum(-np.sqrt(AL*BL/((1-AL)*(1-BL))), -np.sqrt(((1-AL)*(1-BL))/(AL*BL)))
    rho_range = np.arange(np.around(np.amax(min_rho), decimals=1) + 0.1, np.amin(max_rho), step=0.1)
    rho_matrix = np.random.choice(rho_range, size=AL.shape, replace=False)
    return rho_matrix

In [68]:
def rho_sbm_diff_rho(rho_matrix, k, AL, BL, n=100, seed=None):
    AL = symmetrize(AL)
    BL = symmetrize(BL)
    A = sbm([int(n/k)]*k, AL, loops=True)
    AL = np.repeat(np.repeat(AL, n//k, 0), n//k, 1)
    BL = np.repeat(np.repeat(BL, n//k, 0), n//k, 1)
    rho = np.repeat(np.repeat(rho_matrix, n//k, 0), n//k, 1)
    prob = BL + A*rho*np.sqrt((1-AL)*BL*(1-BL)/AL) - (1-A)*rho*np.sqrt(AL*BL*(1-BL)/(1-AL))
    B = np.random.binomial(1, prob)
    B = B.astype(np.float64)
    B = symmetrize(B)
    return A, B

In [63]:
AL = sbm_params(a=0.7, b=0.3)
BL = sbm_params(a=0.2, b=0.5)

In [70]:
rho_matrix = sample_rho_matrix(AL, BL)
A, B = rho_sbm_diff_rho(rho_matrix, 2, AL, BL, seed=0)

In [71]:
rho_matrix

array([[-0.4,  0.2],
       [-0.2,  0.3]])

In [75]:
%%time
n_arr = np.linspace(10, 100, 10, dtype=int)
k = 2
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.7, b=0.3)
rho_matrix = sample_rho_matrix(P1, P2)
print(rho_matrix)
nmc = 500
test_names = ['pearson', 'dcorr', 'mgc']
power_sbm = {
    'pearson': np.zeros_like(n_arr, dtype=float),
    'dcorr': np.zeros_like(n_arr, dtype=float),
    'mgc': np.zeros_like(n_arr, float)
}

for name in test_names:
        for i, n in enumerate(n_arr):
            blocks = np.repeat(np.arange(k), n//k)
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                           rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                         rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                         rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            power_sbm[name][i] = test_power
            print('finish {} for n={}'.format(name, n))

[[0.3 0.8]
 [0.1 0.7]]
finish pearson for n=10
finish pearson for n=20
finish pearson for n=30
finish pearson for n=40
finish pearson for n=50
finish pearson for n=60
finish pearson for n=70
finish pearson for n=80
finish pearson for n=90
finish pearson for n=100
finish dcorr for n=10
finish dcorr for n=20
finish dcorr for n=30
finish dcorr for n=40
finish dcorr for n=50
finish dcorr for n=60
finish dcorr for n=70
finish dcorr for n=80
finish dcorr for n=90
finish dcorr for n=100
finish mgc for n=10
finish mgc for n=20
finish mgc for n=30
finish mgc for n=40
finish mgc for n=50
finish mgc for n=60
finish mgc for n=70
finish mgc for n=80
finish mgc for n=90
finish mgc for n=100
CPU times: user 4min 48s, sys: 1.77 s, total: 4min 50s
Wall time: 4min 51s


In [76]:
power_sbm

{'pearson': array([0.956, 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ,
        1.   ]),
 'dcorr': array([0.938, 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ,
        1.   ]),
 'mgc': array([0.966, 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ,
        1.   ])}

In [77]:
%%time
n_arr = np.linspace(10, 100, 10, dtype=int)
k = 2
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.2, b=0.5)
rho_matrix = sample_rho_matrix(P1, P2)
print(rho_matrix)
nmc = 500
test_names = ['pearson', 'dcorr', 'mgc']
power_sbm_marg = {
    'pearson': np.zeros_like(n_arr, dtype=float),
    'dcorr': np.zeros_like(n_arr, dtype=float),
    'mgc': np.zeros_like(n_arr, float)
}

for name in test_names:
        for i, n in enumerate(n_arr):
            blocks = np.repeat(np.arange(k), n//k)
            if name == 'pearson':
                test = RVCorr(which_test='pearson')
                test_power = pearson_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                           rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'dcorr':
                test = DCorr(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                         rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            elif name == 'mgc':
                test = MGC(compute_distance_matrix=identity)
                test_power = dcorr_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                         rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
            power_sbm_marg[name][i] = test_power
            print('finish {} for n={}'.format(name, n))

[[-0.4  0.2]
 [-0.2  0.3]]
finish pearson for n=10
finish pearson for n=20
finish pearson for n=30
finish pearson for n=40
finish pearson for n=50
finish pearson for n=60
finish pearson for n=70
finish pearson for n=80
finish pearson for n=90
finish pearson for n=100
finish dcorr for n=10
finish dcorr for n=20
finish dcorr for n=30
finish dcorr for n=40
finish dcorr for n=50
finish dcorr for n=60
finish dcorr for n=70
finish dcorr for n=80
finish dcorr for n=90
finish dcorr for n=100
finish mgc for n=10
finish mgc for n=20
finish mgc for n=30
finish mgc for n=40
finish mgc for n=50
finish mgc for n=60
finish mgc for n=70
finish mgc for n=80
finish mgc for n=90
finish mgc for n=100
CPU times: user 4min 47s, sys: 1.77 s, total: 4min 49s
Wall time: 4min 50s


In [78]:
power_sbm_marg

{'pearson': array([0.088, 0.182, 0.444, 0.614, 0.832, 0.9  , 0.964, 0.99 , 0.996,
        1.   ]),
 'dcorr': array([0.07 , 0.162, 0.472, 0.592, 0.79 , 0.864, 0.97 , 0.994, 0.998,
        1.   ]),
 'mgc': array([0.114, 0.24 , 0.322, 0.606, 0.712, 0.926, 0.966, 0.996, 0.998,
        1.   ])}

In [81]:
%%time
n_arr = np.array([10, 50, 100, 300, 500])
k = 2
P1 = sbm_params(a=0.7, b=0.3)
P2 = sbm_params(a=0.7, b=0.3)
rho_matrix = np.array([[0, 0.1], [0, -0.1]])
print(rho_matrix)
nmc = 500
test_names = ['pearson', 'dcorr', 'mgc']
power_sbm1 = {
    'pearson': np.zeros_like(n_arr, dtype=float),
    'dcorr': np.zeros_like(n_arr, dtype=float),
    'mgc': np.zeros_like(n_arr, float)
}

for name in test_names:
    for i, n in enumerate(n_arr):
        blocks = np.repeat(np.arange(k), n//k)
        if name == 'pearson':
            test = RVCorr(which_test='pearson')
            test_power = pearson_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                       rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
        elif name == 'dcorr':
            test = DCorr(compute_distance_matrix=identity)
            test_power = dcorr_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                     rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
        elif name == 'mgc':
            test = MGC(compute_distance_matrix=identity)
            test_power = dcorr_power_two_sided(test, rho_sbm_diff_rho, given_blocks=True, blocks=blocks,
                                     rho_matrix=rho_matrix, AL=P1, BL=P2, k=k, n=n, mc=nmc)
        power_sbm1[name][i] = test_power
        print('finish {} for n={}'.format(name, n))

[[ 0.   0.1]
 [ 0.  -0.1]]
finish pearson for n=10
finish pearson for n=50
finish pearson for n=100
finish pearson for n=300
finish pearson for n=500
finish dcorr for n=10
finish dcorr for n=50
finish dcorr for n=100
finish dcorr for n=300
finish dcorr for n=500
finish mgc for n=10
finish mgc for n=50
finish mgc for n=100
finish mgc for n=300
finish mgc for n=500
CPU times: user 22min 52s, sys: 24.4 s, total: 23min 16s
Wall time: 22min 38s


In [82]:
power_sbm1

{'pearson': array([0.056, 0.104, 0.306, 0.998, 1.   ]),
 'dcorr': array([0.072, 0.1  , 0.342, 0.996, 1.   ]),
 'mgc': array([0.052, 0.084, 0.424, 0.998, 1.   ])}